# Import libs

In [1]:
# Warning control
import warnings
import os
import yaml
from pathlib import Path


from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool

from pydantic import BaseModel, Field
from typing import List, Optional
# from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Customer Request

In [8]:
transcript_content = Path("data/conversation_01.txt").read_text()
print(transcript_content[:300])

CSR: Good morning! Thank you for calling ShiokAh Insurance. My name is Sarah. How may I assist you today?

Customer: Hi Sarah! I want to check about travel insurance leh. Planning to go Japan next month.

CSR: I'll be happy to help you with that. Before we look into the options, could you share some


# Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

## Using FileReadTool

In [ ]:
read_income = FileReadTool(file_path="data/income_travel_insurance_2024.txt")
read_ge = FileReadTool(file_path="data/ge_travel_insurance_2024.txt")

In [ ]:
class CustomerInfo(BaseModel):
    age: int
    medical_conditions: Optional[List[str]] = Field(default_factory=list)
    destination: str
    coverage_factors: List[str] = Field(default_factory=list)
    policy_customization_factors: List[str] = Field(default_factory=list)


class CustomerInfoList(BaseModel):
    customers: List[CustomerInfo] = Field(default_factory=list)